In [32]:
# automatically reload the package when it is modified
%load_ext autoreload
%autoreload 2

In [33]:
from mtr.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from mtr.datasets.waymo.waymo_dataset_bc import WaymoDatasetBC as WaymoDataset
# from mtr.datasets.waymo.waymo_dataset import WaymoDataset

from tools.mtr_env import BatchMTREnv
from tools.mtr_inference import MTRInference
from tools.mtr_lightning import MTR_Lightning
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.distributions import MultivariateNormal, MixtureSameFamily, Categorical, Independent

cfg = cfg_from_yaml_file('tools/cfgs/waymo/bc+10_percent_data.yaml', cfg)
dataset = WaymoDataset(cfg.DATA_CONFIG, training=True)
env = BatchMTREnv(1, dataset)

model = MTR_Lightning(cfg)

# model = model.load_from_checkpoint('output/bc/epoch=49-step=76700.ckpt')
model = model.cuda()

In [ ]:
for key, val in model.named_parameters():
    if val.requires_grad:
        print(key, val.device)

In [ ]:
(model.model.motion_decoder.query_mlps[1].weight.grad!=0).any()

In [34]:
data = dataset[1]


['TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE' 'TYPE_VEHICLE'
 'TYPE_VEHICLE' 'TYPE_PEDESTRIAN' 'TYPE_PEDESTRIAN' 'TYPE_CYCLIST'
 'TYPE_CYCLIST' 'TYPE_CYCLIST' 'TYPE_CYCLIST' 'TYPE_CYCLIST'
 'TYPE_CYCLIST' 'TYPE_VEHICLE']


In [37]:
'TYPE_VEHICLE' != 'TYPE_VEHICLE'

False

In [ ]:
model.zero_grad()
data = dataset[1]
data_batch = dataset.collate_batch([data])

loss = model.training_step(data_batch, 0)
loss.backward()

In [ ]:
print(model.model.motion_decoder.query_mlps[1].weight)

# Test with dataset

In [ ]:
data = dataset[1]
data_batch = dataset.collate_batch([data])
gt = data_batch['input_dict']['center_gt']
mode, mix, gmm, sample = model.sample(data_batch)

In [ ]:
print(model.model.motion_decoder.int_query_embed.requires_grad)

In [ ]:
print(gt)

In [ ]:
print(mode.mean)

# Test in simulation

In [ ]:
env.reset()

In [ ]:
data_batch = env.batch_scene_data
action = model.sample(data_batch)
env.visualize()
print(action)
_ = env.step(action)

In [134]:
from mtr.models.utils import common_layers
import torch

class test(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.query = torch.nn.Parameter(torch.randn((2,4)), requires_grad=True)
        self.model = common_layers.build_mlps(c_in = 4, mlp_channels=[8,4], ret_before_act=True, activation='elu')
        
    def forward(self):
        return self.model(self.query)

model = test()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [138]:
model.zero_grad()
b = model()
loss = torch.abs(b-1).mean()
loss.backward()
optimizer.step()
print(loss, model.query)


tensor(0.9687, grad_fn=<MeanBackward0>) Parameter containing:
tensor([[-0.3499,  0.9735,  0.2286, -0.6830],
        [ 1.1687,  0.1663, -0.0511,  0.4827]], requires_grad=True)
